In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os
import yaml
from datetime import datetime
import glob
import logging
from pathlib import Path


# Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


# Load YAML config for a tonnage
def load_tonnage_config(yaml_path):
    with open(yaml_path, "r") as f:
        cfg = yaml.safe_load(f)
    return cfg.get("case_def_vars", {})


# Build input path from config
def build_input_dir_from_config(cfg):
    folder_type = cfg.get("folder_type")
    tonnage = cfg.get("tonnage")
    sas_url = cfg.get("sas_url", "")

    # First try path from YAML
    base_path = Path(sas_url) / tonnage
    if base_path.exists():
        return str(base_path.resolve())

    # Fallback relative to cwd/data/raw
    rel_path = Path.cwd() / "data" / "raw" / folder_type / tonnage
    if rel_path.exists():
        return str(rel_path.resolve())

    # Final fallback
    return str(base_path)


# Detect probable date/time column in CSV
def detect_date_column(df):
    candidates = ["timestamp", "ts", "time", "event_time", "datetime", "date", "occurred_at", "created_at"]
    for col in df.columns:
        if col.lower() in candidates:
            return col
    # fallback: any datetime type column
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            return col
    # fallback: try object columns
    for col in df.columns:
        if df[col].dtype == "object":
            try:
                pd.to_datetime(df[col], errors="raise")
                return col
            except Exception:
                continue
    return None


# Write pandas DataFrame to Parquet
def write_parquet_table(df, out_path):
    table = pa.Table.from_pandas(df)
    pq.write_table(table, out_path)


# Process CSV to daily parquet
def process_csv_to_daily_parquets(csv_path, tonnage, folder_type, base_output_dir):
    logging.info(f"Processing CSV: {csv_path}")
    df = pd.read_csv(csv_path, low_memory=False)
    date_col = detect_date_column(df)

    if not date_col:
        raise ValueError(f"No date column detected in {csv_path}")
    
    if df[date_col].dtype == 'object':
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    
    df['_date'] = df[date_col].dt.strftime("%Y_%m_%d")

    # Create daily parquet files directly in final output folder
    unique_dates = df['_date'].unique()
    os.makedirs(base_output_dir, exist_ok=True)

    for d in unique_dates:
        df_day = df[df['_date'] == d].drop(columns=['_date'])
        out_file = os.path.join(base_output_dir, f"{tonnage}_{folder_type}_{d}.parquet")
        write_parquet_table(df_day, out_file)
        logging.info(f"[INFO] Saved daily parquet: {out_file}")

    return unique_dates


# Process one tonnage/folder_type
def process_tonnage_historical(yaml_path):
    cfg = load_tonnage_config(yaml_path)
    tonnage = cfg.get("tonnage")
    folder_type = cfg.get("folder_type")
    logging.info(f"Starting processing for tonnage={tonnage}, folder_type={folder_type}")

    input_dir = build_input_dir_from_config(cfg)
    if not os.path.exists(input_dir):
        raise FileNotFoundError(f"Input directory not found: {input_dir}")

    csv_files = sorted(glob.glob(os.path.join(input_dir, "*.csv")))
    if not csv_files:
        raise FileNotFoundError(f"No CSV files found under {input_dir}")

    # Output directory: directly final curated path
    PROJECT_ROOT = Path.cwd().parent.parent
    base_output_dir = PROJECT_ROOT / "data" / "curated" / "enriched" / "daily" / folder_type / tonnage
    os.makedirs(base_output_dir, exist_ok=True)

    # Process each CSV
    for csv_file in csv_files:
        process_csv_to_daily_parquets(csv_file, tonnage, folder_type, str(base_output_dir))

    logging.info(f"Completed processing for {tonnage}/{folder_type}. Final files in: {base_output_dir}")


# Run all YAML configs
if __name__ == "__main__":
    configs_glob = "../../configs/enrich_data_daily_configs/*.yml"
    yaml_files = sorted(glob.glob(configs_glob))
    for y in yaml_files:
        try:
            process_tonnage_historical(y)
        except Exception as e:
            logging.error(f"Failed to process config {y}: {e}")

2025-11-15 23:15:01,050 - INFO - Starting processing for tonnage=const, folder_type=can
2025-11-15 23:15:01,055 - INFO - Processing CSV: D:\Data Science Projects\Machine Learning\fuel-cost-optimization\data\raw\can\const\CONST_can_20230101_20231231.csv
2025-11-15 23:15:18,122 - INFO - [INFO] Saved daily parquet: d:\Data Science Projects\Machine Learning\fuel-cost-optimization\data\curated\enriched\daily\can\const\const_can_2023_01_01.parquet
2025-11-15 23:15:18,221 - INFO - [INFO] Saved daily parquet: d:\Data Science Projects\Machine Learning\fuel-cost-optimization\data\curated\enriched\daily\can\const\const_can_2023_01_02.parquet
2025-11-15 23:15:18,317 - INFO - [INFO] Saved daily parquet: d:\Data Science Projects\Machine Learning\fuel-cost-optimization\data\curated\enriched\daily\can\const\const_can_2023_01_03.parquet
2025-11-15 23:15:18,426 - INFO - [INFO] Saved daily parquet: d:\Data Science Projects\Machine Learning\fuel-cost-optimization\data\curated\enriched\daily\can\const\cons

In [69]:
import pandas as pd

# Read a Parquet file
p_df = pd.read_parquet("../../data/curated/enriched/daily/can/mav28/mav28_can_2023_01_01.parquet")

In [70]:
p_df = p_df.reset_index()

In [71]:
p_df.head()

,index,timestamp,vehicleId,reg_no,vin_no,model,vehicle_type,tonnage,tripId,EngineRPM,...,DPF_LampCommand,OBDLampCommand,EngineAmberLampCommand,AirFilterClogging,WaterInFuelIndicator,EnginePerformanceBiasLevel,EngineBrakeActiveLampCommand,EngineCoolantLevelLowLampCommand,EngineCoolantTempHighLampCommand,EngineRedStopLampCommand
0,0,2023-01-01 09:34:00,V00151,MH96PU4691,TUB74ST7FW1ENW6TN,Tata Signa 2825.TK,Truck,MAV 28,V00151_TR000001,2012,...,Off,Normal,Normal,Normal,NotDetected,Normal,OFF,Normal,Normal,Normal
1,1,2023-01-01 09:35:00,V00151,MH96PU4691,TUB74ST7FW1ENW6TN,Tata Signa 2825.TK,Truck,MAV 28,V00151_TR000001,1353,...,Off,Normal,Normal,Normal,NotDetected,Normal,OFF,Normal,Normal,Normal
2,2,2023-01-01 09:36:00,V00151,MH96PU4691,TUB74ST7FW1ENW6TN,Tata Signa 2825.TK,Truck,MAV 28,V00151_TR000001,2047,...,Off,Normal,Normal,Normal,NotDetected,Economy Mode,OFF,Normal,Normal,Normal
3,3,2023-01-01 09:37:00,V00151,MH96PU4691,TUB74ST7FW1ENW6TN,Tata Signa 2825.TK,Truck,MAV 28,V00151_TR000001,2096,...,Off,Normal,Normal,Normal,NotDetected,Economy Mode,OFF,Normal,Normal,Normal
4,4,2023-01-01 09:38:00,V00151,MH96PU4691,TUB74ST7FW1ENW6TN,Tata Signa 2825.TK,Truck,MAV 28,V00151_TR000001,1721,...,Off,Normal,Normal,Normal,NotDetected,Normal,OFF,Normal,Normal,Normal


In [72]:
p_df.tail()

,index,timestamp,vehicleId,reg_no,vin_no,model,vehicle_type,tonnage,tripId,EngineRPM,...,DPF_LampCommand,OBDLampCommand,EngineAmberLampCommand,AirFilterClogging,WaterInFuelIndicator,EnginePerformanceBiasLevel,EngineBrakeActiveLampCommand,EngineCoolantLevelLowLampCommand,EngineCoolantTempHighLampCommand,EngineRedStopLampCommand
3676,1242399,2023-01-01 20:54:00,V00158,MH57NN4263,CD92UXGXHZ4G451R3,Tata Prima 2830.K,Truck,MAV 28,V00158_TR000004,1752,...,Off,Normal,Normal,Normal,NotDetected,Economy Mode,OFF,Normal,Normal,Normal
3677,1242400,2023-01-01 20:55:00,V00158,MH57NN4263,CD92UXGXHZ4G451R3,Tata Prima 2830.K,Truck,MAV 28,V00158_TR000004,1960,...,Off,Normal,Normal,Normal,NotDetected,Economy Mode,OFF,Normal,Normal,Normal
3678,1242401,2023-01-01 20:56:00,V00158,MH57NN4263,CD92UXGXHZ4G451R3,Tata Prima 2830.K,Truck,MAV 28,V00158_TR000004,2033,...,Off,Normal,Normal,Normal,NotDetected,Economy Mode,OFF,Normal,Normal,Normal
3679,1242402,2023-01-01 20:57:00,V00158,MH57NN4263,CD92UXGXHZ4G451R3,Tata Prima 2830.K,Truck,MAV 28,V00158_TR000004,1547,...,Off,Normal,Normal,Normal,NotDetected,Economy Mode,OFF,Normal,Normal,Normal
3680,1242403,2023-01-01 20:58:00,V00158,MH57NN4263,CD92UXGXHZ4G451R3,Tata Prima 2830.K,Truck,MAV 28,V00158_TR000004,1659,...,Off,Normal,Normal,Normal,NotDetected,Normal,OFF,Normal,Normal,Normal


In [73]:
p_df.shape

(3681, 31)